In [ ]:
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.8 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.0.0
    Uninstalling Werkzeug-3.0.0:
      Successfully uninstalled Werkzeug-3.0.0
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import open3d as o3d

# Load the PLY file
def load_ply(filename):
    pcd = o3d.io.read_point_cloud(filename)
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    return points, colors

root = 'gdrive/My Drive/Proyecto/stereo_servoing/output/'

# Load your point clouds
pcs0, pcc0 = load_ply(root+"point_cloud.ply")
pcs1, pcc1 = load_ply(root+"point_cloud_new.ply")

In [ ]:
def random_subsample(pcs, pcc, sample_size):
    """
    Take a random subsample of the given point cloud and its color.

    :param pcs: Point cloud to be subsampled.
    :param pcc: Colors of the point cloud to be subsampled.
    :param sample_size: Number of points to include in the subsample.
    :return: Subsampled point cloud and its color.
    """
    # Ensure sample size is not greater than the size of the point cloud
    sample_size = min(sample_size, pcs.shape[0])

    # Randomly select indices
    indices = np.random.choice(pcs.shape[0], size=sample_size, replace=False)

    # Extract the subsampled point cloud and color
    subsampled_pcs = pcs[indices]
    subsampled_pcc = pcc[indices]

    return subsampled_pcs, subsampled_pcc

# Test the function on the synthetic data
sample_size = 1000
subsampled_pcs0, subsampled_pcc0 = random_subsample(pcs0, pcc0, sample_size)
subsampled_pcs1, subsampled_pcc1 = random_subsample(pcs1, pcc1, sample_size)

subsampled_pcs0.shape, subsampled_pcc0.shape, subsampled_pcs1.shape, subsampled_pcc1.shape


((1000, 3), (1000, 3), (1000, 3), (1000, 3))

In [ ]:
np.save(root+'s_pcs0', subsampled_pcs0)
np.save(root+'s_pcc0', subsampled_pcc0)
np.save(root+'s_pcs1', subsampled_pcs1)
np.save(root+'s_pcc1', subsampled_pcc1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Load data from files
pcs0 = np.load('/mnt/data/s_pcs0.npy')
pcs1 = np.load('/mnt/data/s_pcs1.npy')
pcc0 = np.load('/mnt/data/s_pcc0.npy') / 255.0  # Normalize to [0, 1]
pcc1 = np.load('/mnt/data/s_pcc1.npy') / 255.0  # Normalize to [0, 1]

# Visualization
fig = plt.figure(figsize=(15, 5))

# Visualize point cloud 0
ax0 = fig.add_subplot(1, 3, 1, projection='3d')
ax0.scatter(pcs0[:, 0], pcs0[:, 1], pcs0[:, 2], c=pcc0, s=5)
ax0.set_title('Point Cloud 0')
ax0.set_xlabel('X')
ax0.set_ylabel('Y')
ax0.set_zlabel('Z')

# Visualize point cloud 1
ax1 = fig.add_subplot(1, 3, 2, projection='3d')
ax1.scatter(pcs1[:, 0], pcs1[:, 1], pcs1[:, 2], c=pcc1, s=5)
ax1.set_title('Point Cloud 1')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')

# Visualize both point clouds together
ax2 = fig.add_subplot(1, 3, 3, projection='3d')
ax2.scatter(pcs0[:, 0], pcs0[:, 1], pcs0[:, 2], c='red', s=5, label='Point Cloud 0')
ax2.scatter(pcs1[:, 0], pcs1[:, 1], pcs1[:, 2], c='blue', s=5, label='Point Cloud 1')
ax2.set_title('Both Point Clouds')
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_zlabel('Z')
ax2.legend()

plt.tight_layout()
plt.show()


In [ ]:
from scipy.spatial import KDTree
from scipy.linalg import svd

def compute_transformation(source, target):
    """
    Compute the transformation to align the source points to the target points.
    """
    # Compute centroids
    centroid_source = np.mean(source, axis=0)
    centroid_target = np.mean(target, axis=0)

    # Center the points
    source_centered = source - centroid_source
    target_centered = target - centroid_target

    # Compute the cross-covariance matrix
    H = np.dot(source_centered.T, target_centered)

    # Compute SVD of H
    U, _, Vt = svd(H)

    # Compute rotation matrix
    R = np.dot(Vt.T, U.T)

    # Ensure a proper rotation
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = np.dot(Vt.T, U.T)

    # Compute translation
    t = centroid_target - np.dot(R, centroid_source)

    return R, t

def colored_icp(source, target, source_colors, target_colors, max_iterations=30, tol=1e-6):
    """
    Colored Iterative Closest Point (CICP) algorithm.
    """
    # Build a KD-tree for target point cloud for efficient nearest neighbor search
    tree = KDTree(target)

    # Initialize transformation
    R = np.eye(3)
    t = np.zeros(3)

    prev_error = float('inf')

    for _ in range(max_iterations):
        # Apply current transformation to the source point cloud
        transformed_source = np.dot(source, R.T) + t

        # Find nearest neighbors in target point cloud
        distances, indices = tree.query(transformed_source)

        # Compute color differences
        color_diffs = np.linalg.norm(source_colors - target_colors[indices], axis=1)

        # Combine geometric and color differences for weighting
        weights = distances + color_diffs

        # Select points based on weights (lower weights are better)
        selected = weights < np.percentile(weights, 80)

        aligned_source = transformed_source[selected]
        aligned_target = target[indices][selected]

        # Compute transformation
        R_new, t_new = compute_transformation(aligned_source, aligned_target)

        # Update transformation
        R = np.dot(R_new, R)
        t = np.dot(R_new, t) + t_new

        # Check for convergence
        error = np.mean(distances[selected])
        if np.abs(prev_error - error) < tol:
            break
        prev_error = error

    return R, t

# Apply CICP
R, t = colored_icp(pcs1, pcs0, pcc1, pcc0)

R, t


In [ ]:
# Apply the estimated transformation to point cloud 1
transformed_pcs1 = np.dot(pcs1, R.T) + t

# Visualization
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Visualize point cloud 0
ax.scatter(pcs0[:, 0], pcs0[:, 1], pcs0[:, 2], c='red', s=5, label='Point Cloud 0 (Target)')

# Visualize transformed point cloud 1
ax.scatter(transformed_pcs1[:, 0], transformed_pcs1[:, 1], transformed_pcs1[:, 2], c='blue', s=5, label='Aligned Point Cloud 1 (Source)')

ax.set_title('Aligned Point Clouds using CICP')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Visualization function for aligned point clouds from different angles
def visualize_aligned_point_clouds(elevation, azimuth):
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    # Visualize point cloud 0
    ax.scatter(pcs0[:, 0], pcs0[:, 1], pcs0[:, 2], c='red', s=5, label='Point Cloud 0 (Target)')

    # Visualize transformed point cloud 1
    ax.scatter(transformed_pcs1[:, 0], transformed_pcs1[:, 1], transformed_pcs1[:, 2], c='blue', s=5, label='Aligned Point Cloud 1 (Source)')

    ax.view_init(elevation, azimuth)

    ax.set_title(f'Aligned Point Clouds (Elevation: {elevation}, Azimuth: {azimuth})')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()

    plt.tight_layout()
    plt.show()

# Display aligned point clouds from different angles
angles = [(30, 30), (60, 30), (90, 30), (30, 60), (60, 90)]
for elevation, azimuth in angles:
    visualize_aligned_point_clouds(elevation, azimuth)
